In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [2]:
# Step 1: Load the historical data
data = pd.read_csv('historical_data.csv')

In [3]:
# Step 2: Filter the required columns and sort by date and ticker
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(['Ticker', 'Date'], inplace=True)

In [4]:
# Step 3: Encode the Ticker column
encoder = OneHotEncoder(sparse=False)
tickers_encoded = encoder.fit_transform(data[['Ticker']])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Step 4: Add the encoded tickers as additional features
encoded_columns = [f'Ticker_{i}' for i in range(tickers_encoded.shape[1])]
encoded_df = pd.DataFrame(tickers_encoded, columns=encoded_columns, index=data.index)
data = pd.concat([data.reset_index(drop=True), encoded_df], axis=1)

In [6]:
# Step 5: Scale numerical features (Open, High, Low, Close, Volume)
scaler = MinMaxScaler(feature_range=(0, 1))
numerical_features = ['Open', 'High', 'Low', 'Close', 'Volume']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [7]:
# Step 6: Preprocessing function
def preprocess_data(data, n_steps=10):
    X, y = [], []
    feature_columns = numerical_features + encoded_columns
    for ticker in data['Ticker'].unique():
        ticker_data = data[data['Ticker'] == ticker] # Ensuring sequences are within the same stock
        ticker_data = ticker_data[feature_columns].values
        for i in range(n_steps, len(ticker_data)):
            X.append(ticker_data[i - n_steps:i, :])  # Last n_steps rows as features
            y.append(ticker_data[i, 3])  # Predict 'Close' price
    return np.array(X), np.array(y)

In [8]:
# Step 7: Preprocess the data
X, y = preprocess_data(data, n_steps=10)

In [9]:
# Step 8: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Step 9: Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  
])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Step 10: Compile the model
model.compile(optimizer='adam', loss='mse')

In [12]:
# Step 11: Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 9.2772e-04 - val_loss: 1.2688e-04
Epoch 2/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 2.2855e-04 - val_loss: 6.5785e-05
Epoch 3/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 1.9754e-04 - val_loss: 9.2746e-05
Epoch 4/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.2502e-04 - val_loss: 3.2200e-05
Epoch 5/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 1.1096e-04 - val_loss: 3.7830e-05
Epoch 6/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 8.9841e-05 - val_loss: 1.2709e-04
Epoch 7/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 1.2018e-04 - val_loss: 5.2774e-05
Epoch 8/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 8.6810e-05 - val_loss: 2.8797e-05
Epoch 9/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 1.2101e-04 - val_loss: 1.0577e-04
Epoch 10/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 8.8737e-05 - val_loss: 1.1067e-04
Epoch 11/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step -

In [13]:
# Step 12: Make predictions
predictions = model.predict(X_test)

236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [14]:
# Step 13: Inverse scale the predictions and actual values
y_test_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((y_test.shape[0], 4)), y_test.reshape(-1, 1)), axis=1))[:, -1]
predictions_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((predictions.shape[0], 4)), predictions), axis=1))[:, -1]

In [21]:
# Step 13: Evaluate the model
mae = mean_absolute_error(predictions_scaled, y_test_scaled)
mape = mean_absolute_percentage_error(predictions_scaled, y_test_scaled)

# Define a percentage threshold for accuracy
threshold_percentage = 5  # 5% tolerance

# Calculate percentage errors
percentage_errors = np.abs((y_test_scaled - predictions_scaled) / y_test_scaled) * 100

# Count predictions within the threshold
acc = np.mean(percentage_errors <= threshold_percentage) * 100

acc2 = 1- mape


In [22]:
# Step 14: Print performance metrics
print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape*100:.2f}%")
print(f"Accuracy = {acc:.2f}%")
print(f"Accuracy = {acc2 *100:.2f}%")

Mean Absolute Error = 4442230.884840585
Mean Absolute Percentage Error = 15.16%
Accuracy = 46.66%
Accuracy = 84.84%
